# 📌 Model–C++ Boyut (Shape) Uyumunu Adam Akıllı Nasıl Kontrol Ederim?

LibTorch tarafında asıl kritik konu şu:

> Python tarafında eğittiğim modelin **beklediği giriş/çıkış şekli** ile  
> C++ tarafında ürettiğim `torch::Tensor`’ın **gerçek şekli** birebir uyumlu mu?

Bunu sağlamanın 3 ana kontratı var:

1. **Input shape kontratı** → `[N, C, H, W]`
2. **Input dtype + scale kontratı** → `float32`, 0–1 veya 0–255 vb.
3. **Output shape kontratı** → `[N, num_classes]` gibi

Aşağıda hem Python hem C++ tarafında bu kontratları nasıl test ettiğimi anlatıyorum.

---

## 🔹 1) Python Tarafında Model IO Kontratını Netleştirme

Önce, Python’da modelimin gerçekten ne beklediğini **bilerek** export ediyorum.

### ✅ Giriş/Çıkış boyutunu test etme

```python
model = SimpleCNN(num_classes=10)
x = torch.randn(1, 3, 64, 64)   # [N=1, C=3, H=64, W=64]
y = model(x)

print("Input shape :", x.shape)   # torch.Size([1, 3, 64, 64])
print("Output shape:", y.shape)   # torch.Size([1, 10])   (örn. CIFAR10)
```
Buradan kendime şunu yazıyorum:

**Giriş: [1, 3, 64, 64], dtype: float32, scale: 0–1**

**Çıkış: [1, 10] (10 sınıf)**

* Bu dokümantasyon gibi düşünebilirsin; C++’ta ne yapacağını buna göre ayarlayacaksın.

### ✅ Dataset tarafını da doğrula
```python
images, labels = next(iter(train_loader))
print("Batch images shape:", images.shape)  # torch.Size([64, 3, 64, 64])
```
Buradan da şunu net görüyorsun:
**transforms.Resize((64,64)) + ToTensor() ile loader → [B, 3, 64, 64] üretiyor.**

>Yani: Python tarafında kontratımız C=3, H=64, W=64.



---
## 🔹 2) TorchScript Export Sonrası da Şekli Yine Python’da Test Et

* model_ts.pt’yi kaydettikten sonra, C++’a geçmeden önce bir defa daha Python’da test edebilirsin:
```python
ts_model = torch.jit.load("model_ts.pt")
ts_model.eval()

x = torch.randn(1, 3, 64, 64)
y = ts_model(x)

print("TS Input shape :", x.shape)
print("TS Output shape:", y.shape)
```


Burada da [1, 3, 64, 64] -> [1, 10] görüyorsan:

**Modelin TorchScript versiyonu da aynı IO kontratını kullanıyor demektir.**
----

---

## 🔹 3) C++ Tarafında Preprocess Sonrası Shape’i Direkt Yazdır

C++’ta asıl kritik nokta:
**OpenCV → Tensor dönüşümü sonrası tensörün gerçekten [1, 3, 64, 64] olup olmadığını kontrol etmek.**

Preprocess fonksiyonundan sonra şunu yaparım:
```cpp
torch::Tensor input_tensor = preprocess_image(img_bgr, 64);
std::cout << "[DEBUG] Input tensor sizes: " << input_tensor.sizes() << std::endl;
```

LibTorch şu şekilde basar:
```bash
[DEBUG] Input tensor sizes: [1, 3, 64, 64]
```


Eğer şunu görürsem:

* [1, 64, 64, 3] → NHWC kalmışım, permute yapmayı unutmuşum

* [3, 64, 64] → batch dimension yok

* [1, 1, 64, 64] → grayscale gibi

Hemen preprocess fonksiyonuna geri döner ve BGR→RGB, permute, batch dimension kısımlarını düzeltirim.


### İstersen runtime assert koy
```cpp
auto sizes = input_tensor.sizes();
if (!(sizes.size() == 4 &&
      sizes[0] == 1 &&
      sizes[1] == 3 &&
      sizes[2] == 64 &&
      sizes[3] == 64)) {
    std::cerr << "[HATA] Input tensor shape beklenen formatta degil! "
              << "Gelen: " << sizes << std::endl;
    return -1;
}
```


**Bu, “şartlar sağlanmıyorsa programı direkt patlat” kontratı.**

---
## 🔹 4) Forward Sonrası Output Shape’i de C++ Tarafında Yazdır

Modelin çıkışının da beklediğim gibi olduğunu C++’ta görmek için:
```cpp
torch::Tensor output = module.forward(inputs).toTensor();
std::cout << "[DEBUG] Output tensor sizes: " << output.sizes() << std::endl;
```


Beklediğim:

>[DEBUG] Output tensor sizes: [1, 10]


Eğer:

* [10] → batch dimension düşmüş

* [1, 1000] → yanlış model (ImageNet pretrained vs.)

* [1, 4] → belki başka bir num_classes ile train ettin

Hemen Python tarafına dönüp:

* num_classes değerini

* C++ tarafında nasıl yorumladığını

kontrol ederim.

----
## 🔹 5) dtype ve normalize kontratını da unutma

* Sadece shape yetmiyor; dtype + scale de uyumlu olmalı.

Python tarafı (dataset + training) için:
```python
print(images.dtype)     # torch.float32
print(images.min(), images.max())  # tensor(0.0) tensor(1.0)  (ToTensor ile)
```


C++ tarafında benzerini tensör için kontrol edebilirsin:
```cpp
std::cout << "dtype: " << input_tensor.dtype() << std::endl;
// is_floating_point, min/max gibi kontroller için istersen CPU'ya çekip bakabilirsin
```


Kısaca:

* Python ToTensor → [0,1] float32 ise

**C++ preprocess’de de convertTo(..., CV_32F, 1.0/255.0) yapıp aynı ölçeği yakalarsın.**

---

## 🔹 6) Mismatch Olduğunda Zaten TorchScript Patlatır

* Bir yerde unuttun diyelim, yine de kör gitmiyorsun; TorchScript içerde şekli beğenmezse:

>c10::Error fırlatır,

>Biz zaten C++’ta try { ... } catch (const c10::Error& e) yazıyoruz.

Bu sayede:
```cpp
catch (const c10::Error& e) {
    std::cerr << "[EXCEPTION - c10] " << e.what() << std::endl;
}
```


* içinde sana “beklenen kanal sayısı şu, gelen bu” tarzı hata mesajları verir.

----

## 🔚 Özet Kontrat Mantığı

### Python’da:

* x = torch.randn(1,3,64,64) → y = model(x) → shape’leri yaz.

* next(iter(train_loader)) ile gerçek batch shape’ini yaz.

* TorchScript’te de (torch.jit.load) aynı testi yap.

### C++’ta:

* Preprocess sonrası input_tensor.sizes() → konsola yaz.

* Runtime check ile [1,3,64,64] değilse hata bas.

* Forward sonrası output.sizes() → [1,10] olduğunu doğrula.

* dtype + normalize kontratını da düşün:

* Python: ToTensor → 0–1 float32

*  C++: CV_32F, 1.0/255.0 ile aynı davranışı yakala.

----